<a href="https://colab.research.google.com/github/ViviAkhemi/1First_LLM/blob/main/First_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
!pip install -q -U google-generativeai

In [47]:
# importações e configurações iniciais
import numpy as np
import pandas as pd
import google.generativeai as genai
from google.colab import userdata
api_key = userdata.get('SECRETA_CHAVE')
#GOOGLE_API_KEY=('SECRETA_CHAVE')
genai.configure(api_key=api_key)

In [18]:
for m in genai.list_models():
  if 'embedContent' in m.supported_generation_methods:
    print(m.name)

models/embedding-001
models/text-embedding-004


In [19]:
#Exemplo de embedding
title = "A próxima geração de IA para desenvolvedores e Google Workspace"
sample_text = ("Título: A próxima geração de IA para desenvolvedores e Google Workspace"
    "\n"
    "Artigo completo:\n"
    "\n"
    "Gemini API & Google AI Studio: Uma maneira acessível de explorar e criar protótipos com aplicaões de IA gerenerativa")
embeddings = genai.embed_content(model="models/embedding-001",
                                content=sample_text,
                                title=title,
                                task_type="RETRIEVAL_DOCUMENT")
print(embeddings)

{'embedding': [0.048038475, -0.030389505, -0.021319741, 0.021499945, 0.063412175, -0.009704742, -0.030610476, -0.0112727685, 0.06302876, 0.059729308, 0.012074996, 0.007864701, -0.040817786, -0.02574182, 0.006324538, -0.02588424, 0.027180783, -0.014828148, -0.029813053, -0.0039829686, 0.0022281483, 0.006926729, -0.030743396, -0.062132377, -0.020105815, 0.018456187, 0.01819799, -0.0395632, -0.031490624, 0.024801038, -0.05280905, 0.047006182, -0.032477852, 0.012259007, -0.04054164, -0.04173661, -0.02581353, -0.047700617, -0.0053919265, -0.0031763988, 0.007353981, -0.08565834, -0.010035877, 0.027569804, -0.0049378686, -0.02756122, 0.04113799, 0.053774305, 0.020193119, -0.055419285, 0.025774838, 0.024286203, 0.07005263, -0.04093201, -0.00012558069, -0.0073715067, 0.030554753, -0.039339133, 0.02329524, -0.0016537032, 0.0017721604, 0.016804215, -0.016996942, 0.05913256, 0.021242557, -0.05741046, -0.04548596, -0.0111473715, 0.015385055, 0.048099186, 0.014034349, 0.013910542, 0.056646492, -0.06

In [8]:
#Listagem de documentos que serão buscados
DOCUMENT1 = {
    "Título": "Operação do sistema de controle climático",
    "Conteúdo": "O Googlecar tem um sistema de controle climático que permite ajusta a temperatura e o fluxo de ar no carro."
    }

DOCUMENT2 = {
    "Título": "Touchscreen",
    "Conteúdo": "O seu Googlecar tem uma grande tela sensível ao toque que fornece acesso a uma variedade de recursos, incluindo navegação."
    }
DOCUMENT3 = {
    "Título": "Mudança de marchas",
    "Conteúdo": "Seu Googlecar tem uma transmissão automática. Para trocar as marchas, basta mover a alavanca de câmbio."
}

documents = [DOCUMENT1, DOCUMENT2, DOCUMENT3]


In [11]:
df = pd.DataFrame(documents)
df.columns = ["Título", "Conteúdo"]
df

,Título,Conteúdo
0,Operação do sistema de controle climático,O Googlecar tem um sistema de controle climáti...
1,Touchscreen,O seu Googlecar tem uma grande tela sensível a...
2,Mudança de marchas,Seu Googlecar tem uma transmissão automática. ...


In [30]:
model = "models/embedding-001"

In [37]:
def embed_fn(title, text):
  return genai.embed_content(model=model,
                             content=text,
                             title=title,
                             task_type="RETRIEVAL_DOCUMENT")["embedding"]

In [38]:
df["Embeddings"] = df.apply(lambda row: embed_fn(row["Título"], row["Conteúdo"]), axis=1)
df

,Título,Conteúdo,Embeddings
0,Operação do sistema de controle climático,O Googlecar tem um sistema de controle climáti...,"[-0.009241557, -0.017497232, -0.061312307, 0.0..."
1,Touchscreen,O seu Googlecar tem uma grande tela sensível a...,"[0.011367341, -0.048442047, -0.023845239, -0.0..."
2,Mudança de marchas,Seu Googlecar tem uma transmissão automática. ...,"[0.0035545328, -0.040299606, -0.0060573816, 0...."


In [40]:
def gerar_e_buscar_consulta(consulta, base, model):
  embedding_da_consulta = genai.embed_content(model=model,
                                              content=consulta,
                                              task_type="RETRIEVAL_QUERY")["embedding"]
  produtos_escalares = np.dot(np.stack(df["Embeddings"]), embedding_da_consulta)

  indice = np.argmax(produtos_escalares)
  return df.iloc[indice]["Conteúdo"]

In [41]:
consulta = "Como faço para trocar marchas em um carro do Google?"

trecho = gerar_e_buscar_consulta(consulta, df, model)
print(trecho)

Seu Googlecar tem uma transmissão automática. Para trocar as marchas, basta mover a alavanca de câmbio.


In [42]:
generation_config = {
    "temperature": 0,
    "candidate_count": 1,
}

In [44]:
prompt = f"Reescreva esse texto de uma forma mais descontraída, sem adicionar informações que não façam parte do texto: {trecho}"

model_2 = genai.GenerativeModel("gemini-1.0-pro",
                                generation_config=generation_config)
response = model_2.generate_content(prompt)
print(response.text)


Seu Googlecar é tipo um carro normal, só que sem volante! Mas não se preocupe, você ainda pode trocar as marchas de boa. É só dar uma mexida na alavanca de câmbio, tipo quando você dirige um carro de verdade.
